# **Modeling with MLflow**

This notebook trains and compares multiple regression models to predict Airbnb listing prices using the V2 preprocessed dataset. All experiments are tracked using MLflow to enable systematic comparison and model selection.

The preprocessed V2 datasets include neighbourhood target encoding and interaction features for improved predictive power.

**Modeling strategy:**
1. Baseline models with default parameters
2. Log transformation experiments for linear models (to handle skewed price distribution)
3. Hyperparameter tuning for top performers
4. Final model selection and registration

## Setup and Data Loading

In [1]:
import pandas as pd
import numpy as np
import boto3
import io
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load V2 preprocessed data from S3

bucket_name = 'software-tools-ai'
s3_client = boto3.client('s3')

# Load train data
train_key = 'processed_data/train_processed_v2.csv'
response = s3_client.get_object(Bucket=bucket_name, Key=train_key)
train_df = pd.read_csv(io.BytesIO(response['Body'].read()))

# Load test data
test_key = 'processed_data/test_processed_v2.csv'
response = s3_client.get_object(Bucket=bucket_name, Key=test_key)
test_df = pd.read_csv(io.BytesIO(response['Body'].read()))

# Separate features and target
X_train = train_df.drop('price', axis=1)
y_train = train_df['price']

X_test = test_df.drop('price', axis=1)
y_test = test_df['price']

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nFeature count: {X_train.shape[1]}")
print(f"\nPrice range - Train: ${y_train.min():.0f} to ${y_train.max():.0f}")
print(f"Price range - Test: ${y_test.min():.0f} to ${y_test.max():.0f}")

Training set: (38715, 29)
Test set: (9777, 29)

Feature count: 29

Price range - Train: $10 to $795
Price range - Test: $10 to $8000


In [3]:
# Connect to MLflow server on EC2
mlflow.set_tracking_uri('http://35.183.177.64:5000')

# Create/use experiment
mlflow.set_experiment('airbnb-price-prediction')

print("MLflow setup complete")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment: {mlflow.get_experiment_by_name('airbnb-price-prediction').name}")

MLflow setup complete
Tracking URI: http://35.183.177.64:5000
Experiment: airbnb-price-prediction


In [4]:
# Helper function to evaluate models consistently

def evaluate_model(y_true, y_pred, model_name="Model"):
    """
    Calculate regression metrics.
    Evaluates on comparable price range (<$795) to avoid outlier distortion.
    """
    # Full test set metrics
    rmse_full = np.sqrt(mean_squared_error(y_true, y_pred))
    mae_full = mean_absolute_error(y_true, y_pred)
    r2_full = r2_score(y_true, y_pred)
    
    # Comparable range metrics (price <= $795, which is ~99% of test data)
    mask = y_true <= 795
    rmse_comparable = np.sqrt(mean_squared_error(y_true[mask], y_pred[mask]))
    mae_comparable = mean_absolute_error(y_true[mask], y_pred[mask])
    r2_comparable = r2_score(y_true[mask], y_pred[mask])
    
    print(f"\n{model_name} Performance:")
    print(f"  Full test set    - RMSE: ${rmse_full:.2f}, MAE: ${mae_full:.2f}, R²: {r2_full:.3f}")
    print(f"  Comparable range - RMSE: ${rmse_comparable:.2f}, MAE: ${mae_comparable:.2f}, R²: {r2_comparable:.3f}")
    
    return {
        'rmse_full': rmse_full,
        'mae_full': mae_full,
        'r2_full': r2_full,
        'rmse_comparable': rmse_comparable,
        'mae_comparable': mae_comparable,
        'r2_comparable': r2_comparable
    }

## Baseline Models

Train baseline models with reasonable default parameters to establish performance benchmarks.

In [5]:
# Linear Regression - Baseline

with mlflow.start_run(run_name="linear_regression_v2"):
    # Train
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    # Predict
    predictions = lr.predict(X_test)
    
    # Evaluate
    metrics = evaluate_model(y_test, predictions, "Linear Regression")
    
    # Log to MLflow
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("dataset_version", "v2")
    mlflow.log_param("transform", "none")
    
    mlflow.log_metric("rmse_full", metrics['rmse_full'])
    mlflow.log_metric("mae_full", metrics['mae_full'])
    mlflow.log_metric("r2_full", metrics['r2_full'])
    mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
    mlflow.log_metric("mae_comparable", metrics['mae_comparable'])
    mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
    
    mlflow.sklearn.log_model(lr, "model")


Linear Regression Performance:
  Full test set    - RMSE: $186.43, MAE: $62.83, R²: 0.132
  Comparable range - RMSE: $79.68, MAE: $49.97, R²: 0.396


2025/11/23 09:39:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:39:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run linear_regression_v2 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/3c3e1600e15040a0bb728c5279c2bb05
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


In [6]:
# Ridge Regression - Baseline (alpha=1.0)

with mlflow.start_run(run_name="ridge_v2_alpha1"):
    # Train
    ridge = Ridge(alpha=1.0, random_state=42)
    ridge.fit(X_train, y_train)
    
    # Predict
    predictions = ridge.predict(X_test)
    
    # Evaluate
    metrics = evaluate_model(y_test, predictions, "Ridge (alpha=1.0)")
    
    # Log to MLflow
    mlflow.log_param("model_type", "Ridge")
    mlflow.log_param("alpha", 1.0)
    mlflow.log_param("dataset_version", "v2")
    mlflow.log_param("transform", "none")
    
    mlflow.log_metric("rmse_full", metrics['rmse_full'])
    mlflow.log_metric("mae_full", metrics['mae_full'])
    mlflow.log_metric("r2_full", metrics['r2_full'])
    mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
    mlflow.log_metric("mae_comparable", metrics['mae_comparable'])
    mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
    
    mlflow.sklearn.log_model(ridge, "model")


Ridge (alpha=1.0) Performance:
  Full test set    - RMSE: $186.43, MAE: $62.83, R²: 0.132
  Comparable range - RMSE: $79.68, MAE: $49.97, R²: 0.396


2025/11/23 09:40:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:40:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ridge_v2_alpha1 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/b36bdff6253a4ce388438e9274a06fea
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


In [7]:
# Lasso Regression - Baseline (alpha=1.0)

with mlflow.start_run(run_name="lasso_v2_alpha1"):
    # Train
    lasso = Lasso(alpha=1.0, random_state=42, max_iter=10000)
    lasso.fit(X_train, y_train)
    
    # Predict
    predictions = lasso.predict(X_test)
    
    # Evaluate
    metrics = evaluate_model(y_test, predictions, "Lasso (alpha=1.0)")
    
    # Log to MLflow
    mlflow.log_param("model_type", "Lasso")
    mlflow.log_param("alpha", 1.0)
    mlflow.log_param("dataset_version", "v2")
    mlflow.log_param("transform", "none")
    
    # Check feature selection
    n_features_used = np.sum(lasso.coef_ != 0)
    mlflow.log_metric("n_features_selected", n_features_used)
    print(f"\nFeatures selected by Lasso: {n_features_used}/{len(lasso.coef_)}")
    
    mlflow.log_metric("rmse_full", metrics['rmse_full'])
    mlflow.log_metric("mae_full", metrics['mae_full'])
    mlflow.log_metric("r2_full", metrics['r2_full'])
    mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
    mlflow.log_metric("mae_comparable", metrics['mae_comparable'])
    mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
    
    mlflow.sklearn.log_model(lasso, "model")


Lasso (alpha=1.0) Performance:
  Full test set    - RMSE: $186.73, MAE: $62.95, R²: 0.129
  Comparable range - RMSE: $80.01, MAE: $50.07, R²: 0.391

Features selected by Lasso: 9/29


2025/11/23 09:40:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:40:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run lasso_v2_alpha1 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/f97ced4fcb1b49e69c761c463efd09fd
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


In [17]:
# Random Forest - Baseline

with mlflow.start_run(run_name="random_forest_v2_baseline"):
    # Train
    rf = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    # Predict
    predictions = rf.predict(X_test)
    
    # Evaluate
    metrics = evaluate_model(y_test, predictions, "Random Forest")
    
    # Log to MLflow
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 20)
    mlflow.log_param("dataset_version", "v2")
    
    mlflow.log_metric("rmse_full", metrics['rmse_full'])
    mlflow.log_metric("mae_full", metrics['mae_full'])
    mlflow.log_metric("r2_full", metrics['r2_full'])
    mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
    mlflow.log_metric("mae_comparable", metrics['mae_comparable'])
    mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
    
    # mlflow.sklearn.log_model(rf, "model")


Random Forest Performance:
  Full test set    - RMSE: $180.91, MAE: $57.54, R²: 0.183
  Comparable range - RMSE: $73.89, MAE: $45.00, R²: 0.480
🏃 View run random_forest_v2_baseline at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/9a6ba9f1cdb94fcabe21ded940abe461
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


In [8]:
# XGBoost - Baseline

with mlflow.start_run(run_name="xgboost_v2_baseline"):
    # Train
    xgb = XGBRegressor(n_estimators=100, max_depth=7, learning_rate=0.1, random_state=42, n_jobs=-1)
    xgb.fit(X_train, y_train)
    
    # Predict
    predictions = xgb.predict(X_test)
    
    # Evaluate
    metrics = evaluate_model(y_test, predictions, "XGBoost")
    
    # Log to MLflow
    mlflow.log_param("model_type", "XGBoost")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 7)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("dataset_version", "v2")
    
    mlflow.log_metric("rmse_full", metrics['rmse_full'])
    mlflow.log_metric("mae_full", metrics['mae_full'])
    mlflow.log_metric("r2_full", metrics['r2_full'])
    mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
    mlflow.log_metric("mae_comparable", metrics['mae_comparable'])
    mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
    
    mlflow.sklearn.log_model(xgb, "model")


XGBoost Performance:
  Full test set    - RMSE: $180.39, MAE: $56.88, R²: 0.187
  Comparable range - RMSE: $72.85, MAE: $44.33, R²: 0.495


2025/11/23 09:40:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:40:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run xgboost_v2_baseline at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/f71a6196735e49b395446a2af93ae98f
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


### Baseline Results Summary

After running all baseline models, review the MLflow UI to identify top performers before proceeding to log transformation experiments.

## Log Transformation Experiments

Linear models assume a linear relationship between features and target. However, Airbnb prices are right-skewed (many cheap listings, few expensive ones). Log transformation compresses this distribution, potentially improving linear model performance.

**Why log transform helps:**
- Reduces impact of outliers without removing them
- Makes relationship more linear
- Stabilizes variance (homoscedasticity)

**Note:** Tree-based models (RF, XGBoost) don't need this transformation as they're already robust to skewed distributions.

In [9]:
# Transform target variable to log scale

# Using log1p = log(1 + x) to handle edge cases
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

print(f"Original price range: ${y_train.min():.2f} to ${y_train.max():.2f}")
print(f"Log-transformed range: {y_train_log.min():.2f} to {y_train_log.max():.2f}")
print(f"\nLog transformation compresses the scale and reduces outlier impact")

Original price range: $10.00 to $795.00
Log-transformed range: 2.40 to 6.68

Log transformation compresses the scale and reduces outlier impact


In [10]:
# Linear Regression + Log Transform

with mlflow.start_run(run_name="linear_v2_log_transform"):
    # Train on log scale
    lr_log = LinearRegression()
    lr_log.fit(X_train, y_train_log)
    
    # Predict on log scale
    predictions_log = lr_log.predict(X_test)
    
    # Convert back to original scale
    predictions = np.expm1(predictions_log)  # expm1 = exp(x) - 1
    
    # Evaluate on original scale
    metrics = evaluate_model(y_test, predictions, "Linear Regression + Log")
    
    # Log to MLflow
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("dataset_version", "v2")
    mlflow.log_param("transform", "log")
    
    mlflow.log_metric("rmse_full", metrics['rmse_full'])
    mlflow.log_metric("mae_full", metrics['mae_full'])
    mlflow.log_metric("r2_full", metrics['r2_full'])
    mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
    mlflow.log_metric("mae_comparable", metrics['mae_comparable'])
    mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
    
    mlflow.sklearn.log_model(lr_log, "model")


Linear Regression + Log Performance:
  Full test set    - RMSE: $188.16, MAE: $59.60, R²: 0.116
  Comparable range - RMSE: $80.90, MAE: $46.54, R²: 0.377


2025/11/23 09:40:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:40:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run linear_v2_log_transform at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/7d52ea2ed6cb402f897aadf42165c972
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


In [11]:
# Ridge + Log Transform

with mlflow.start_run(run_name="ridge_v2_log_alpha1"):
    # Train on log scale
    ridge_log = Ridge(alpha=1.0, random_state=42)
    ridge_log.fit(X_train, y_train_log)
    
    # Predict and convert back
    predictions_log = ridge_log.predict(X_test)
    predictions = np.expm1(predictions_log)
    
    # Evaluate
    metrics = evaluate_model(y_test, predictions, "Ridge + Log (alpha=1.0)")
    
    # Log to MLflow
    mlflow.log_param("model_type", "Ridge")
    mlflow.log_param("alpha", 1.0)
    mlflow.log_param("dataset_version", "v2")
    mlflow.log_param("transform", "log")
    
    mlflow.log_metric("rmse_full", metrics['rmse_full'])
    mlflow.log_metric("mae_full", metrics['mae_full'])
    mlflow.log_metric("r2_full", metrics['r2_full'])
    mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
    mlflow.log_metric("mae_comparable", metrics['mae_comparable'])
    mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
    
    mlflow.sklearn.log_model(ridge_log, "model")


Ridge + Log (alpha=1.0) Performance:
  Full test set    - RMSE: $188.16, MAE: $59.60, R²: 0.116
  Comparable range - RMSE: $80.90, MAE: $46.54, R²: 0.377


2025/11/23 09:41:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:41:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ridge_v2_log_alpha1 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/6df8747490544f4e99980cc7dcc9c7e0
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


In [12]:
# Lasso + Log Transform

with mlflow.start_run(run_name="lasso_v2_log_alpha1"):
    # Train on log scale
    lasso_log = Lasso(alpha=1.0, random_state=42, max_iter=10000)
    lasso_log.fit(X_train, y_train_log)
    
    # Predict and convert back
    predictions_log = lasso_log.predict(X_test)
    predictions = np.expm1(predictions_log)
    
    # Evaluate
    metrics = evaluate_model(y_test, predictions, "Lasso + Log (alpha=1.0)")
    
    # Log to MLflow
    mlflow.log_param("model_type", "Lasso")
    mlflow.log_param("alpha", 1.0)
    mlflow.log_param("dataset_version", "v2")
    mlflow.log_param("transform", "log")
    
    n_features_used = np.sum(lasso_log.coef_ != 0)
    mlflow.log_metric("n_features_selected", n_features_used)
    print(f"\nFeatures selected by Lasso: {n_features_used}/{len(lasso_log.coef_)}")
    
    mlflow.log_metric("rmse_full", metrics['rmse_full'])
    mlflow.log_metric("mae_full", metrics['mae_full'])
    mlflow.log_metric("r2_full", metrics['r2_full'])
    mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
    mlflow.log_metric("mae_comparable", metrics['mae_comparable'])
    mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
    
    mlflow.sklearn.log_model(lasso_log, "model")


Lasso + Log (alpha=1.0) Performance:
  Full test set    - RMSE: $204.17, MAE: $82.76, R²: -0.041
  Comparable range - RMSE: $105.91, MAE: $69.25, R²: -0.068

Features selected by Lasso: 0/29


2025/11/23 09:41:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:41:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run lasso_v2_log_alpha1 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/ed05058c1fa1485f8a1d2c474b4a8e4e
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


## Hyperparameter Tuning

Based on baseline results, tune hyperparameters for the top-performing models. This section focuses on the models that showed the most promise in initial experiments.

### Ridge/Lasso Alpha Tuning

Alpha controls regularization strength. Test different values to find optimal balance between bias and variance.

In [13]:
# Ridge alpha tuning

alphas = [0.1, 10.0, 100.0]

for alpha in alphas:
    with mlflow.start_run(run_name=f"ridge_v2_alpha{alpha}"):
        # Train
        ridge = Ridge(alpha=alpha, random_state=42)
        ridge.fit(X_train, y_train)
        
        # Predict
        predictions = ridge.predict(X_test)
        
        # Evaluate
        metrics = evaluate_model(y_test, predictions, f"Ridge (alpha={alpha})")
        
        # Log to MLflow
        mlflow.log_param("model_type", "Ridge")
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("dataset_version", "v2")
        mlflow.log_param("transform", "none")
        
        mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
        mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
        mlflow.log_metric("rmse_full", metrics['rmse_full'])
        mlflow.log_metric("r2_full", metrics['r2_full'])
        
        mlflow.sklearn.log_model(ridge, "model")


Ridge (alpha=0.1) Performance:
  Full test set    - RMSE: $186.43, MAE: $62.83, R²: 0.132
  Comparable range - RMSE: $79.68, MAE: $49.97, R²: 0.396


2025/11/23 09:41:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:41:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ridge_v2_alpha0.1 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/eeac96b7833f4087a48297131ff788c7
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579

Ridge (alpha=10.0) Performance:
  Full test set    - RMSE: $186.43, MAE: $62.82, R²: 0.132
  Comparable range - RMSE: $79.67, MAE: $49.96, R²: 0.396


2025/11/23 09:41:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:41:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ridge_v2_alpha10.0 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/294c9f2e8db743f4ac1af9d685e1f4c9
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579

Ridge (alpha=100.0) Performance:
  Full test set    - RMSE: $186.45, MAE: $62.79, R²: 0.132
  Comparable range - RMSE: $79.67, MAE: $49.93, R²: 0.396


2025/11/23 09:41:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:41:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ridge_v2_alpha100.0 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/4258b0334e95460eb4816232b6ae37d4
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


### Random Forest Tuning

Tune n_estimators, max_depth, and min_samples_split for optimal performance.

In [16]:
# Random Forest tuning

rf_configs = [
    {'n_estimators': 200, 'max_depth': 20, 'min_samples_split': 2},
    {'n_estimators': 100, 'max_depth': 25, 'min_samples_split': 2},
    {'n_estimators': 200, 'max_depth': 25, 'min_samples_split': 5},
]

for config in rf_configs:
    run_name = f"rf_v2_n{config['n_estimators']}_d{config['max_depth']}_s{config['min_samples_split']}"
    
    with mlflow.start_run(run_name=run_name):
        # Train
        rf = RandomForestRegressor(
            n_estimators=config['n_estimators'],
            max_depth=config['max_depth'],
            min_samples_split=config['min_samples_split'],
            random_state=42,
            n_jobs=-1
        )
        rf.fit(X_train, y_train)
        
        # Predict
        predictions = rf.predict(X_test)
        
        # Evaluate
        metrics = evaluate_model(y_test, predictions, f"RF {config}")
        
        # Log to MLflow
        mlflow.log_param("model_type", "RandomForest")
        mlflow.log_param("n_estimators", config['n_estimators'])
        mlflow.log_param("max_depth", config['max_depth'])
        mlflow.log_param("min_samples_split", config['min_samples_split'])
        mlflow.log_param("dataset_version", "v2")
        
        mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
        mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
        mlflow.log_metric("rmse_full", metrics['rmse_full'])
        mlflow.log_metric("r2_full", metrics['r2_full'])
        
        # mlflow.sklearn.log_model(rf, "model")


RF {'n_estimators': 200, 'max_depth': 20, 'min_samples_split': 2} Performance:
  Full test set    - RMSE: $180.76, MAE: $57.41, R²: 0.184
  Comparable range - RMSE: $73.75, MAE: $44.88, R²: 0.482
🏃 View run rf_v2_n200_d20_s2 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/81f212bb13e2491faa299928d63b9b23
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579

RF {'n_estimators': 100, 'max_depth': 25, 'min_samples_split': 2} Performance:
  Full test set    - RMSE: $180.74, MAE: $57.73, R²: 0.184
  Comparable range - RMSE: $74.08, MAE: $45.22, R²: 0.478
🏃 View run rf_v2_n100_d25_s2 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/bc86a04d05bb42b8b69d15f3ec8eca61
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579

RF {'n_estimators': 200, 'max_depth': 25, 'min_samples_split': 5} Performance:
  Full test set    - RMSE: $180.68, MAE: $57.49, R²: 0.185
  Comparable range - RMSE: $73.76, MAE: $44.9

### XGBoost Tuning

Tune learning_rate, max_depth, and n_estimators for gradient boosting optimization.

In [14]:
# XGBoost tuning

xgb_configs = [
    {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.1},
    {'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.1},
    {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.05},
]

for config in xgb_configs:
    run_name = f"xgb_v2_n{config['n_estimators']}_d{config['max_depth']}_lr{config['learning_rate']}"
    
    with mlflow.start_run(run_name=run_name):
        # Train
        xgb = XGBRegressor(
            n_estimators=config['n_estimators'],
            max_depth=config['max_depth'],
            learning_rate=config['learning_rate'],
            random_state=42,
            n_jobs=-1
        )
        xgb.fit(X_train, y_train)
        
        # Predict
        predictions = xgb.predict(X_test)
        
        # Evaluate
        metrics = evaluate_model(y_test, predictions, f"XGB {config}")
        
        # Log to MLflow
        mlflow.log_param("model_type", "XGBoost")
        mlflow.log_param("n_estimators", config['n_estimators'])
        mlflow.log_param("max_depth", config['max_depth'])
        mlflow.log_param("learning_rate", config['learning_rate'])
        mlflow.log_param("dataset_version", "v2")
        
        mlflow.log_metric("rmse_comparable", metrics['rmse_comparable'])
        mlflow.log_metric("r2_comparable", metrics['r2_comparable'])
        mlflow.log_metric("rmse_full", metrics['rmse_full'])
        mlflow.log_metric("r2_full", metrics['r2_full'])
        
        mlflow.sklearn.log_model(xgb, "model")


XGB {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.1} Performance:
  Full test set    - RMSE: $180.01, MAE: $56.79, R²: 0.191
  Comparable range - RMSE: $72.79, MAE: $44.27, R²: 0.496


2025/11/23 09:41:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:41:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run xgb_v2_n200_d7_lr0.1 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/40fd11a5e85a4013afe811d02140c751
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579

XGB {'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.1} Performance:
  Full test set    - RMSE: $180.47, MAE: $56.79, R²: 0.186
  Comparable range - RMSE: $72.86, MAE: $44.21, R²: 0.495


2025/11/23 09:42:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:42:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run xgb_v2_n100_d9_lr0.1 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/ddc8175785564ea39199bcc2ef146577
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579

XGB {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.05} Performance:
  Full test set    - RMSE: $181.59, MAE: $57.58, R²: 0.176
  Comparable range - RMSE: $73.50, MAE: $44.95, R²: 0.486


2025/11/23 09:42:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 09:42:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run xgb_v2_n200_d5_lr0.05 at: http://35.183.177.64:5000/#/experiments/279774441150129579/runs/5b78a65c9ab04a13bddfdf60beaea1cd
🧪 View experiment at: http://35.183.177.64:5000/#/experiments/279774441150129579


## Model Selection and Registration

Review all experiments in MLflow UI, select the best model based on r2_comparable metric, and register it for deployment.

In [ ]:
# Access MLflow UI to review all experiments
print("Review all experiments at: http://35.183.177.64:5000")
print("\nSort runs by 'r2_comparable' metric (descending) to find best model")
print("\nNext steps:")
print("1. Identify best performing model")
print("2. Register model in MLflow Model Registry")
print("3. Take screenshots for documentation")

## Key Findings

**To be completed after reviewing MLflow results:**

**Best model:**
- Model type: [TBD]
- RMSE (comparable): [TBD]
- R² (comparable): [TBD]

**Key insights:**
- Impact of neighbourhood target encoding on model performance
- Effectiveness of log transformation for linear models
- Comparison between linear and tree-based approaches
- Dataset limitations (R² ceiling around 0.50 due to missing features like bedrooms, bathrooms, amenities)

**Next steps:**
- Register best model in MLflow
- Document findings in README
- Prepare final presentation materials